In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt 
import pickle
import fastf1
import os



In [2]:
device = ("cuda:0" if torch.cuda.is_available else "cpu")
print(device)

cuda:0


In [3]:
data = pd.read_csv(open('data/Data_with_encoding.csv','rb'),delimiter=',')
print(data.head)

<bound method NDFrame.head of       Year  Round           Driver          Team   AvgFPTime  QualyTime  \
0     2022      1  Charles Leclerc       Ferrari  110.585000     90.558   
1     2022      1     Carlos Sainz       Ferrari  109.985100     90.687   
2     2022      1   Lewis Hamilton      Mercedes  110.977098     91.048   
3     2022      1   George Russell      Mercedes  107.507673     91.252   
4     2022      1  Kevin Magnussen  Haas F1 Team  111.297087     91.461   
...    ...    ...              ...           ...         ...        ...   
1534  2025      9     Esteban Ocon  Haas F1 Team   96.682667     73.201   
1535  2025      9   Oliver Bearman  Haas F1 Team   97.584178     73.074   
1536  2025      9   Kimi Antonelli      Mercedes   90.285085     72.111   
1537  2025      9  Alexander Albon      Williams   94.480767     72.641   
1538  2024      3   Logan Sargeant      Williams  102.000000     78.000   

      QualTimeDelta  GridPos  FinishPos IsAccurate  Team_encoded  \
0

In [4]:
save_dir = 'scaling_params'
mean_vals_path = os.path.join(save_dir, 'mean_vals.pt')
std_vals_path = os.path.join(save_dir, 'std_vals.pt')

In [5]:
loaded_mean_vals = torch.load(mean_vals_path)
loaded_std_vals = torch.load(std_vals_path)
loaded_mean_vals = loaded_mean_vals.to(device)
loaded_std_vals = loaded_std_vals.to(device)
print(loaded_std_vals.device)
print(loaded_mean_vals.device)

cuda:0
cuda:0


In [6]:
DRIVER_ENCODER_PATH = 'encoders/driver_encoder.pkl'
TEAM_ENCODER_PATH = 'encoders/team_encoder.pkl'

try:
    with open(DRIVER_ENCODER_PATH, 'rb') as f:
        loaded_driver_encoder = pickle.load(f)
    print(f"Driver encoder loaded from {DRIVER_ENCODER_PATH}")

    with open(TEAM_ENCODER_PATH, 'rb') as f:
        loaded_team_encoder = pickle.load(f)
    print(f"Team encoder loaded from {TEAM_ENCODER_PATH}")
except FileNotFoundError:
    print("Error: Encoder files not found. Make sure they are in the correct directory.")
    exit()

Driver encoder loaded from encoders/driver_encoder.pkl
Team encoder loaded from encoders/team_encoder.pkl


In [7]:
num_drivers = len(loaded_driver_encoder.classes_)
num_teams = len(loaded_team_encoder.classes_)
print(loaded_driver_encoder.classes_)
print(loaded_team_encoder.classes_)

['Alexander Albon' 'Andrea Kimi Antonelli' 'Carlos Sainz'
 'Charles Leclerc' 'Daniel Ricciardo' 'Esteban Ocon' 'Fernando Alonso'
 'Franco Colapinto' 'Gabriel Bortoleto' 'George Russell' 'Guanyu Zhou'
 'Isack Hadjar' 'Jack Doohan' 'Kevin Magnussen' 'Kimi Antonelli'
 'Lance Stroll' 'Lando Norris' 'Lewis Hamilton' 'Liam Lawson'
 'Logan Sargeant' 'Max Verstappen' 'Mick Schumacher' 'Nicholas Latifi'
 'Nico Hulkenberg' 'Nyck De Vries' 'Oliver Bearman' 'Oscar Piastri'
 'Pierre Gasly' 'Sebastian Vettel' 'Sergio Perez' 'Valtteri Bottas'
 'Yuki Tsunoda']
['Alpine' 'Aston Martin' 'Ferrari' 'Haas F1 Team' 'Kick Sauber' 'McLaren'
 'Mercedes' 'Racing Bulls' 'Red Bull Racing' 'Williams']


In [8]:
from F1_Net import F1Net


new_net = F1Net(num_teams,num_drivers)
new_net.to(device)


F1Net(
  (team_embedding): Embedding(10, 4)
  (driver_embedding): Embedding(32, 4)
  (fc1): Linear(in_features=12, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [9]:
model_path = "saved_models/f1_ranking_model_new_2.pth"

try:
    new_net.load_state_dict(torch.load(model_path,map_location=device))
except FileNotFoundError:
    print("Modle File not found")
    exit()

# New Races


In [10]:
import os
cache_folder = 'cache_folder'

if not os.path.exists(cache_folder):
    os.makedirs(cache_folder)

fastf1.Cache.enable_cache(cache_folder)

In [11]:
#Get Data For the Upcoming Race
round_num = 12
year = 2025

In [12]:
all_fp_laps = []

# Load FP1 (required)
sessionFP1 = fastf1.get_session(year, round_num, 'FP1')
sessionFP1.load()
all_fp_laps.append(sessionFP1.laps)

# Try FP2 and FP3
for fp_type in ['FP2', 'FP3']:
    try:
        session = fastf1.get_session(year, round_num, fp_type)
        session.load()
        all_fp_laps.append(session.laps)
    except ValueError:
        pass


all_fp_laps = pd.concat(all_fp_laps, ignore_index=True)

core           INFO 	Loading data for British Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '5', '6', '10', '12', '14', '16', '18', '23', '30', '31', '36', '43', '44', '55', '63', '81', '87', '9

In [13]:
avg_fp = all_fp_laps.groupby('Driver')["LapTime"].mean().dt.total_seconds()



In [14]:
sessionQ = fastf1.get_session(year, round_num, 'Q')
sessionQ.load()

results = sessionQ.results.set_index("Abbreviation")

#Get Qualy deltas

q_laps_fastest = sessionQ.laps.pick_fastest()
q_laps = sessionQ.laps.pick_not_deleted()

if isinstance(q_laps,pd.Series):
    q_laps = q_laps.to_frame().T

fastest_time = q_laps_fastest["LapTime"]

valid_laps = q_laps[q_laps["LapTime"].notna()]

fastest_laps = valid_laps.loc[valid_laps.groupby('Driver')['LapTime'].idxmin()]

fastest_laps["LapDelta"] = fastest_laps["LapTime"].apply(lambda x: x-fastest_time).dt.total_seconds()

fastest_laps["LapTime"] = fastest_laps["LapTime"].dt.total_seconds()
fastest_laps["FullName"] =  fastest_laps["Driver"].map(results["FullName"])


core           INFO 	Loading data for British Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '81', '4', '63', '44', '16', '12', '87', '14', '10', '55', '22', '6', '23', '31', '30', '5', '18', '27', '43']


In [15]:
fastest_laps

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,LapDelta,FullName
235,0 days 00:53:46.517000,ALB,23,85.889,9.0,4.0,NaT,NaT,0 days 00:00:27.803000,0 days 00:00:34.456000,...,0 days 00:52:20.628000,2025-07-05 14:38:46.369,1,NaN,False,,False,True,0.997,Alexander Albon
161,0 days 01:02:59.575000,ALO,14,85.593,8.0,3.0,NaT,NaT,0 days 00:00:27.645000,0 days 00:00:34.365000,...,0 days 01:01:33.982000,2025-07-05 14:47:59.723,1,NaN,False,,False,True,0.701,Fernando Alonso
134,0 days 01:22:52.585000,ANT,12,85.374,17.0,6.0,NaT,NaT,0 days 00:00:27.658000,0 days 00:00:34.279000,...,0 days 01:21:27.211000,2025-07-05 15:07:52.952,1,NaN,False,,False,True,0.482,Kimi Antonelli
152,0 days 01:23:06.901000,BEA,87,85.471,17.0,6.0,NaT,NaT,0 days 00:00:27.698000,0 days 00:00:34.297000,...,0 days 01:21:41.430000,2025-07-05 15:08:07.171,1,NaN,False,,False,True,0.579,Oliver Bearman
265,0 days 00:38:25.921000,BOR,5,86.446,8.0,3.0,NaT,NaT,0 days 00:00:27.991000,0 days 00:00:34.778000,...,0 days 00:36:59.475000,2025-07-05 14:23:25.216,1,NaN,False,,False,True,1.554,Gabriel Bortoleto
286,0 days 00:17:40.550000,COL,43,87.060,2.0,1.0,NaT,NaT,0 days 00:00:28.281000,0 days 00:00:35.024000,...,0 days 00:16:13.490000,2025-07-05 14:02:39.231,1,NaN,False,,False,True,2.168,Franco Colapinto
182,0 days 01:04:02.471000,GAS,10,85.711,14.0,4.0,NaT,NaT,0 days 00:00:27.777000,0 days 00:00:34.533000,...,0 days 01:02:36.760000,2025-07-05 14:49:02.501,1,NaN,False,,False,True,0.819,Pierre Gasly
225,0 days 01:03:46.255000,HAD,6,85.864,11.0,4.0,NaT,NaT,0 days 00:00:27.906000,0 days 00:00:34.458000,...,0 days 01:02:20.391000,2025-07-05 14:48:46.132,1,NaN,False,,False,True,0.972,Isack Hadjar
89,0 days 01:02:51.270000,HAM,44,85.084,12.0,4.0,NaT,NaT,0 days 00:00:27.528000,0 days 00:00:34.179000,...,0 days 01:01:26.186000,2025-07-05 14:47:51.927,1,NaN,False,,False,True,0.192,Lewis Hamilton
283,0 days 00:38:18.640000,HUL,27,86.574,8.0,3.0,NaT,NaT,0 days 00:00:28.176000,0 days 00:00:34.820000,...,0 days 00:36:52.066000,2025-07-05 14:23:17.807,1,NaN,False,,False,True,1.682,Nico Hulkenberg


In [16]:
fastest_laps_sorted = fastest_laps.sort_values("LapTime").reset_index(drop=True)
fastest_laps_sorted["Position"] = fastest_laps_sorted.index + 1  # 1-based grid

fastest_laps_sorted

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,LapDelta,FullName
0,0 days 01:24:27.741000,VER,1,84.892,17.0,6.0,NaT,NaT,0 days 00:00:27.493000,0 days 00:00:33.973000,...,0 days 01:23:02.849000,2025-07-05 15:09:28.590,1,1,False,,False,True,0.000,Max Verstappen
1,0 days 01:15:20.045000,PIA,81,84.995,17.0,6.0,NaT,NaT,0 days 00:00:27.615000,0 days 00:00:34.113000,...,0 days 01:13:55.050000,2025-07-05 15:00:20.791,1,2,False,,False,True,0.103,Oscar Piastri
2,0 days 01:23:17.659000,NOR,4,85.010,19.0,6.0,NaT,NaT,0 days 00:00:27.659000,0 days 00:00:34.027000,...,0 days 01:21:52.649000,2025-07-05 15:08:18.390,1,3,False,,False,True,0.118,Lando Norris
3,0 days 01:23:00.708000,RUS,63,85.029,18.0,6.0,NaT,NaT,0 days 00:00:27.644000,0 days 00:00:34.163000,...,0 days 01:21:35.679000,2025-07-05 15:08:01.420,1,4,False,,False,True,0.137,George Russell
4,0 days 01:02:51.270000,HAM,44,85.084,12.0,4.0,NaT,NaT,0 days 00:00:27.528000,0 days 00:00:34.179000,...,0 days 01:01:26.186000,2025-07-05 14:47:51.927,1,5,False,,False,True,0.192,Lewis Hamilton
5,0 days 01:23:44.870000,LEC,16,85.121,20.0,6.0,NaT,NaT,0 days 00:00:27.635000,0 days 00:00:34.036000,...,0 days 01:22:19.749000,2025-07-05 15:08:45.490,1,6,False,,False,True,0.229,Charles Leclerc
6,0 days 01:22:52.585000,ANT,12,85.374,17.0,6.0,NaT,NaT,0 days 00:00:27.658000,0 days 00:00:34.279000,...,0 days 01:21:27.211000,2025-07-05 15:07:52.952,1,7,False,,False,True,0.482,Kimi Antonelli
7,0 days 01:23:06.901000,BEA,87,85.471,17.0,6.0,NaT,NaT,0 days 00:00:27.698000,0 days 00:00:34.297000,...,0 days 01:21:41.430000,2025-07-05 15:08:07.171,1,8,False,,False,True,0.579,Oliver Bearman
8,0 days 01:02:59.575000,ALO,14,85.593,8.0,3.0,NaT,NaT,0 days 00:00:27.645000,0 days 00:00:34.365000,...,0 days 01:01:33.982000,2025-07-05 14:47:59.723,1,9,False,,False,True,0.701,Fernando Alonso
9,0 days 01:04:02.471000,GAS,10,85.711,14.0,4.0,NaT,NaT,0 days 00:00:27.777000,0 days 00:00:34.533000,...,0 days 01:02:36.760000,2025-07-05 14:49:02.501,1,10,False,,False,True,0.819,Pierre Gasly


In [38]:
# List of all drivers you want to include (from Qualifying session or FP)
drivers = fastest_laps["Driver"].unique()

data = []
try:
    sessionR = fastf1.get_session(year, round_num, 'R')
    sessionR.load()
    results = sessionR.results.set_index("Abbreviation")  # Use abbreviation (e.g. 'VER', 'LEC') for matching
    grid_available = True
except:
    results = None
    grid_available = False

for driver_code in drivers:
    fp_time = avg_fp.get(driver_code)
    q_data = fastest_laps_sorted[fastest_laps_sorted["Driver"] == driver_code]

    if not q_data.empty:
        qualyDelta = q_data.iloc[0]["LapDelta"]
        qualyTime = q_data.iloc[0]["LapTime"]
        is_accurate = q_data.iloc[0]["IsAccurate"]
        team_name = q_data.iloc[0]["Team"]
        driver_name = q_data.iloc[0]["FullName"]
    else:
        qualyDelta = None
        qualyTime = None
        is_accurate = None
        team_name = None
        driver_name = None

    # Use qualifying position if available, or set as None
    if grid_available and driver_code in results.index:
        grid_pos = results.loc[driver_code]["GridPosition"]
        print("Grid Source = Race")
    else:
        grid_pos = q_data.iloc[0]["Position"] if not q_data.empty else None
        print("Grid Source = Qualifying")

    data.append({
        "Year": year,
        "Round": round_num,
        "Driver": driver_name,
        "Team": team_name,
        "AvgFPTime": fp_time,
        "QualyTime": qualyTime,
        "QualTimeDelta": qualyDelta,
        "GridPos": grid_pos,
        "FinishPos": None,  # No race data yet
        "IsAccurate": is_accurate
    })

new_race = pd.DataFrame(data)


core           INFO 	Loading data for British Grand Prix - Race [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	Failed to load driver list and session results!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
logger      WARNING 	Failed to load total lap count!
req        

Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying
Grid Source = Qualifying


In [39]:
new_race

,Year,Round,Driver,Team,AvgFPTime,QualyTime,QualTimeDelta,GridPos,FinishPos,IsAccurate
0,2025,12,Alexander Albon,Williams,105.003831,85.889,0.997,14,None,True
1,2025,12,Fernando Alonso,Aston Martin,104.725578,85.593,0.701,9,None,True
2,2025,12,Kimi Antonelli,Mercedes,102.130489,85.374,0.482,7,None,True
3,2025,12,Oliver Bearman,Haas F1 Team,100.806244,85.471,0.579,8,None,True
4,2025,12,Gabriel Bortoleto,Kick Sauber,103.365833,86.446,1.554,17,None,True
5,2025,12,Franco Colapinto,Alpine,99.701239,87.060,2.168,20,None,True
6,2025,12,Pierre Gasly,Alpine,108.070818,85.711,0.819,10,None,True
7,2025,12,Isack Hadjar,Racing Bulls,104.787786,85.864,0.972,13,None,True
8,2025,12,Lewis Hamilton,Ferrari,104.813312,85.084,0.192,5,None,True
9,2025,12,Nico Hulkenberg,Kick Sauber,104.859282,86.574,1.682,19,None,True


In [40]:
new_race['Team_encoded'] = -1
new_race['Driver_encoded'] = -1

new_race

,Year,Round,Driver,Team,AvgFPTime,QualyTime,QualTimeDelta,GridPos,FinishPos,IsAccurate,Team_encoded,Driver_encoded
0,2025,12,Alexander Albon,Williams,105.003831,85.889,0.997,14,None,True,-1,-1
1,2025,12,Fernando Alonso,Aston Martin,104.725578,85.593,0.701,9,None,True,-1,-1
2,2025,12,Kimi Antonelli,Mercedes,102.130489,85.374,0.482,7,None,True,-1,-1
3,2025,12,Oliver Bearman,Haas F1 Team,100.806244,85.471,0.579,8,None,True,-1,-1
4,2025,12,Gabriel Bortoleto,Kick Sauber,103.365833,86.446,1.554,17,None,True,-1,-1
5,2025,12,Franco Colapinto,Alpine,99.701239,87.060,2.168,20,None,True,-1,-1
6,2025,12,Pierre Gasly,Alpine,108.070818,85.711,0.819,10,None,True,-1,-1
7,2025,12,Isack Hadjar,Racing Bulls,104.787786,85.864,0.972,13,None,True,-1,-1
8,2025,12,Lewis Hamilton,Ferrari,104.813312,85.084,0.192,5,None,True,-1,-1
9,2025,12,Nico Hulkenberg,Kick Sauber,104.859282,86.574,1.682,19,None,True,-1,-1


In [41]:
known_drivers = set(loaded_driver_encoder.classes_)
known_teams = set(loaded_team_encoder.classes_)

print(known_drivers,known_teams)




for index,row in new_race.iterrows():
    driver_name = row["Driver"]
    if(driver_name in known_drivers):
        new_race.loc[index,'Driver_encoded'] = loaded_driver_encoder.transform([driver_name])[0]


for index,row in new_race.iterrows():
    team_name = row["Team"]

    if team_name in known_teams:
        new_race.loc[index,'Team_encoded'] = loaded_team_encoder.transform([team_name])[0]




{'Mick Schumacher', 'Gabriel Bortoleto', 'Logan Sargeant', 'Pierre Gasly', 'Fernando Alonso', 'Sergio Perez', 'Esteban Ocon', 'Kevin Magnussen', 'Nyck De Vries', 'Nico Hulkenberg', 'Liam Lawson', 'Oliver Bearman', 'Isack Hadjar', 'Lance Stroll', 'Daniel Ricciardo', 'Kimi Antonelli', 'Lando Norris', 'Charles Leclerc', 'Nicholas Latifi', 'Franco Colapinto', 'Carlos Sainz', 'Oscar Piastri', 'Andrea Kimi Antonelli', 'Sebastian Vettel', 'Valtteri Bottas', 'Guanyu Zhou', 'Max Verstappen', 'Yuki Tsunoda', 'Jack Doohan', 'George Russell', 'Alexander Albon', 'Lewis Hamilton'} {'Williams', 'McLaren', 'Aston Martin', 'Red Bull Racing', 'Ferrari', 'Haas F1 Team', 'Mercedes', 'Alpine', 'Kick Sauber', 'Racing Bulls'}


In [42]:
print("\nnew_race_df with encoded columns:\n")
new_race


new_race_df with encoded columns:



,Year,Round,Driver,Team,AvgFPTime,QualyTime,QualTimeDelta,GridPos,FinishPos,IsAccurate,Team_encoded,Driver_encoded
0,2025,12,Alexander Albon,Williams,105.003831,85.889,0.997,14,None,True,9,0
1,2025,12,Fernando Alonso,Aston Martin,104.725578,85.593,0.701,9,None,True,1,6
2,2025,12,Kimi Antonelli,Mercedes,102.130489,85.374,0.482,7,None,True,6,14
3,2025,12,Oliver Bearman,Haas F1 Team,100.806244,85.471,0.579,8,None,True,3,25
4,2025,12,Gabriel Bortoleto,Kick Sauber,103.365833,86.446,1.554,17,None,True,4,8
5,2025,12,Franco Colapinto,Alpine,99.701239,87.060,2.168,20,None,True,0,7
6,2025,12,Pierre Gasly,Alpine,108.070818,85.711,0.819,10,None,True,0,27
7,2025,12,Isack Hadjar,Racing Bulls,104.787786,85.864,0.972,13,None,True,7,11
8,2025,12,Lewis Hamilton,Ferrari,104.813312,85.084,0.192,5,None,True,2,17
9,2025,12,Nico Hulkenberg,Kick Sauber,104.859282,86.574,1.682,19,None,True,4,23


In [43]:
new_race_tensor = torch.tensor(new_race[["AvgFPTime","QualyTime","QualTimeDelta","GridPos","Team_encoded","Driver_encoded"]].values).float()
new_race_tensor = new_race_tensor.to(device)

numerical_feature_indices = [0, 1, 2, 3]

new_race_norm = (new_race_tensor.clone()).unsqueeze(0)



new_race_norm[:, :, numerical_feature_indices] = \
    (new_race_norm[:, :, numerical_feature_indices] - loaded_mean_vals) / loaded_std_vals


new_race_norm = new_race_norm.squeeze()
print("\nNew race data normalized using loaded parameters.")
print(new_race_norm.shape)


New race data normalized using loaded parameters.
torch.Size([20, 6])


In [44]:
print(new_race_tensor.shape)

new_net.eval()

team_idx = new_race_tensor[:,4].long()
driver_idx = new_race_tensor[:,5].long()
features = new_race_tensor[:,:4]

scores = new_net(team_idx,driver_idx,features)

torch.Size([20, 6])


In [46]:
if scores.dim() > 1 and scores.shape[1] == 1:
    scores = scores.squeeze(1)


driver_names = new_race['Driver'].tolist()


scores_list = scores.tolist()


driver_scores = list(zip(driver_names, scores_list))


driver_scores.sort(key=lambda x: x[1], reverse=True)


print(f"---------{sessionQ.session_info['Meeting']['Name']} Results--------------")

results_df = pd.DataFrame({
    'Driver': driver_names,
    'Score': scores_list
})
results_df = results_df.sort_values(by='Score', ascending=True).reset_index(drop=True)

print("Position\tDriver")
result_list = results_df["Driver"].tolist()

for pos,driver in enumerate(result_list):
    print(f"{pos+1}\t\t{driver}")

---------British Grand Prix Results--------------
Position	Driver
1		Lando Norris
2		Oscar Piastri
3		Max Verstappen
4		Lewis Hamilton
5		Oliver Bearman
6		George Russell
7		Charles Leclerc
8		Fernando Alonso
9		Kimi Antonelli
10		Pierre Gasly
11		Carlos Sainz
12		Yuki Tsunoda
13		Esteban Ocon
14		Isack Hadjar
15		Liam Lawson
16		Alexander Albon
17		Gabriel Bortoleto
18		Lance Stroll
19		Franco Colapinto
20		Nico Hulkenberg


In [57]:
sessionR = fastf1.get_session(year,round_num,'R')
sessionR.load()


results = sessionR.results.set_index("DriverNumber")






core           INFO 	Loading data for British Grand Prix - Race [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	Failed to load driver list and session results!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
logger      WARNING 	Failed to load total lap count!
req        

In [ ]:
#Create Rows
final_results = []
for drv in results.index:
    row = results.loc[drv]
    #print(row)
    cur_driver_name = row["FullName"]
    final_results.append(cur_driver_name)
    cur_finish_pos = row["Position"]
    starting_grid_pos = row["GridPosition"]
    #print(cur_driver_name,cur_finish_pos)
    new_race.loc[new_race["Driver"] == cur_driver_name, ["FinishPos", "GridPos"]] = cur_finish_pos,starting_grid_pos
    
    

In [110]:
print("-"*17,sessionR.session_info["Meeting"]["Name"],"-"*17)
print("{:<10} {:<25} {:<25}".format("Rank", "Predicted Driver", "Actual Driver"))
print("-" * 60)
i = 0
for predicted,actual in zip(result_list,final_results):
    print(f"{i+1:<10} {predicted:<25} {actual:<25}")
    i+=1

----------------- Austrian Grand Prix -----------------
Rank       Predicted Driver          Actual Driver            
------------------------------------------------------------
1          Lando Norris              Lando Norris             
2          Oscar Piastri             Oscar Piastri            
3          Lewis Hamilton            Charles Leclerc          
4          Charles Leclerc           Lewis Hamilton           
5          Gabriel Bortoleto         George Russell           
6          George Russell            Liam Lawson              
7          Pierre Gasly              Fernando Alonso          
8          Max Verstappen            Gabriel Bortoleto        
9          Liam Lawson               Nico Hulkenberg          
10         Fernando Alonso           Esteban Ocon             
11         Isack Hadjar              Oliver Bearman           
12         Kimi Antonelli            Isack Hadjar             
13         Oliver Bearman            Pierre Gasly             
1

In [56]:
new_race

,Year,Round,Driver,Team,AvgFPTime,QualyTime,QualTimeDelta,GridPos,FinishPos,IsAccurate,Team_encoded,Driver_encoded
0,2025,12,Alexander Albon,Williams,105.003831,85.889,0.997,12,17.0,True,9,0
1,2025,12,Fernando Alonso,Aston Martin,104.725578,85.593,0.701,11,7.0,True,1,6
2,2025,12,Kimi Antonelli,Mercedes,102.130489,85.374,0.482,9,19.0,True,6,14
3,2025,12,Oliver Bearman,Haas F1 Team,100.806244,85.471,0.579,15,11.0,True,3,25
4,2025,12,Gabriel Bortoleto,Kick Sauber,103.365833,86.446,1.554,8,8.0,True,4,8
5,2025,12,Franco Colapinto,Alpine,99.701239,87.060,2.168,14,15.0,True,0,7
6,2025,12,Pierre Gasly,Alpine,108.070818,85.711,0.819,10,13.0,True,0,27
7,2025,12,Isack Hadjar,Racing Bulls,104.787786,85.864,0.972,13,12.0,True,7,11
8,2025,12,Lewis Hamilton,Ferrari,104.813312,85.084,0.192,4,4.0,True,2,17
9,2025,12,Nico Hulkenberg,Kick Sauber,104.859282,86.574,1.682,20,9.0,True,4,23


In [ ]:
#print(new_race)
full_data = pd.read_csv(open("data/combined_driver_data_cleaned.csv","rb"),delimiter=",") #Use Combined_driver_data_updated after is has been created
#print(full_data)
update = new_race.drop(["Team_encoded","Driver_encoded"],axis = 1)
df_updated = pd.concat([full_data, update],ignore_index=True)

df_updated.to_csv("data/Combined_driver_data_updated.csv",index=False)
